## 라이브러리 Import

In [1]:
import numpy as np
import random
import os
import math

from glob import glob
# https://wikidocs.net/83
import pandas as pd
import cv2
from tqdm.auto import tqdm
# https://frhyme.github.io/python-libs/python_tqdm/

import torch
import torch.nn as nn # 신경망 호출
import torch.nn.functional as F #nn과 똑같지만 nn은 클래스로 정의되고 functional은 함수로 정의됨
from torch.utils.data import DataLoader, Dataset
#데이터 로딩을 위한 클래스 https://huffon.github.io/2020/05/26/torch-data/

import torchvision.models as models
from torchvision import transforms
#https://better-tomorrow.tistory.com/entry/TorchVision-model-funetuning

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# 쿠다를 사용할수 있으면 쿠다를 사용하고 아니면 CPU로 연산

## 하이퍼파라미터 세팅 Hyperparameters Setting

In [3]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':110,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':8,
    'SEED':41
}

경사 하강법 유형에는 대표적으로 배치경사하강법, 확률적 경사하강법(SGD) 미니배치경사하강법이 존재 요즘 많이 사용하는 미니배치경사하강법은 전체 데이터셋을 미니 배치라는 여러개로 나누고, 미니 배치 한 개마다 기울기를 구한 후 그것의 평균 기울기를 이용하여 모델을 업데이터해서 학습하는 방법. 데이터 전체를 사용해서 하강법을 계산하는 것 보다 빠르고 SGD보다 안정적인 것이 장점

## Fix RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(CFG['SEED']) # Seed 고정

왜 시드를 고정하는가? 난수(랜덤 한 데이터) 생성 시, 그 값은 생성할 때마다 바뀌게 된다. 데이터 셋이 바뀌게 되면, 일관된 결과를 얻기가 힘들어, 제대로 된 비교가 힘들어지므로, 난수를 생성하는 방식을 고정한다. 이를 시드 결정(Set seed)이라 하며, 아무 숫자나 넣어도 상관없다.

## Data Pre-processing 데이터 전처리 함수 설정

In [5]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    # 이미지와 라벨 빈 리스트 생성
    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list.extend(label_df['leaf_weight'])
                
    return img_path_list, label_list
    # 이미지데이터와 라벨 데이터 분리

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0])) 
    return img_path_list

In [6]:
all_img_path, all_label = get_train_data('./dataset/train')
test_img_path = get_test_data('./dataset/test')

all_img_path <- 학습할 이미지 데이터  
all_label <- 이미지 데이터의 라벨(1일 뒤의 잎의 너비)  
test_img_path <- 테스트할 이미지 데이터  

## Train / Validation Split
훈련데이터와 검증 데이터 분리

In [7]:
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(all_img_path)*0.8)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

## CustomDataset

In [8]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        # 이 단계에서 이미지를 원하는데로 전처리 하는 것이 가능하다! 
        img = cv2.imread(img_path)
        img_bgr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cropped = img_bgr[50:-50, 100:-100]
        #ret, image = cv2.threshold(cropped, 120, 255, cv2.THRESH_BINARY_INV)
        #ret, image = cv2.threshold(cropped, 120, 255, cv2.THRESH_BINARY)
        ret, image = cv2.threshold(cropped, 120, 255, cv2.THRESH_TOZERO_INV)
        if self.transforms is not None:
            image = self.transforms(image)
        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [9]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
                    transforms.RandomVerticalFlip(0.2), # 상하 대칭
                    #transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])
test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                    ])
#이미지와 테스트 테이터를 어떻게 변환을 할까? 텐서로 하고, 우리가 지정한 이미지 크기로 리사이즈를 하고 정규화를 시행한다
#https://wikidocs.net/157285 transforms 변수에 대한 설명

In [10]:
# Get Dataloader
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
#batch_size여기서 배치사이즈의 크기를 결정

vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
# 훈련데이터와 검증 데이터를 각각 분리해서 넣는다

## Define Model Architecture

In [11]:
class CNNRegressor(torch.nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.layer1 = torch.nn.Sequential(
            #[8,3,256,256] -> [8,8,256,256]
            nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU(),
            #[8,256,256] -> [8,128,128]
            nn.MaxPool2d(kernel_size=2, stride=2))
            
            #[8,8,256,256] -> [8,16,128,128]
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            #[16,128,128]-> [16,64,64]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,16,64,64] -> [8,32,64,64]
        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            #[8,32,64,64]-> [8,32,32,32]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,32,32,32] -> [8,64,32,32]
        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            #[8,64,32,32] -> [8,64,16,16]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
            #[8,64,16,16] -> [8,128,16,16]
        self.layer5 = torch.nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            #[8,128,16,16] -> [8,128,7,7]
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.regressor=nn.Linear(128*7*7,1)
        # 보통 여기서는 fc, full conneted network으로 불리는 선형출력
        # nn.Linear()는 입력의 차원, 출력의 차원 레이어를 만들어 본다.
        # 이런 레이어 대신 VGG나 ResNet으로하면 성능 향상이 더 있을까?

    def forward(self, x):
        # Simple CNN Model (Batch, 3, 128, 128 -> Batch, 64, 7, 7)
        # (Batch, 1, 256, 256)
        x = self.layer1(x)
        # (Batch, 8, 128, 128)
        x = self.layer2(x)
        # (Batch, 16, 64, 64)
        x = self.layer3(x)
        # (Batch, 32, 32, 32)
        x = self.layer4(x)
        # (Batch, 64, 16, 16)
        x = self.layer5(x)
        # (Batch, 128, 7, 7) -> Flatten (Batch, 128*7*7)
        x = torch.flatten(x, start_dim=1)
        # Regressor (Batch, 128*7*7) -> (Batch, 1)
        out = self.regressor(x)
        return out

torch.nn.Sequential: 순차 모델이라 하며, 레이어를 선형으로 연결해 구성한다. 일반적으로 사용하는 모델로 하나의 텐서가 입력되고 출력되는 단일 입력, 단일 출력에 사용된다. 다중 입력, 다중 출력을 하는 경우나, 레이어를 공유하는 등의 경우엔 사용하지 않는다.

## Train

In [12]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)
    # Loss Function
    criterion = nn.L1Loss().to(device)
    #손실 함수를 어떻게 설정할까? 일단 L1Loss는 MAE(입력의 각 요소 간의 평균 절대 오차)이다.
    # https://pytorch.org/docs/stable/nn.html#loss-functions
    best_mae = 9999
    #기본적으로 엉터리 값을 주고 시작을 해서 점점 낮은 곳으로 가게 한다.
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        #0부터 해도 상관없겠지만 가시성때문에 1부터 시작하는대신 설정한 Epoch에 1을 더한듯?
        model.train()
        train_loss = [] #이거는 전파/역전파에 사용하는 로스
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()
            # 매번 학습이 완료하면(즉, Iteration이 한번 끝나면) gradients를 항상 0으로 만들어 주어야함

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            loss.backward() #역전파 시작
            optimizer.step() # 기울기업데이트
            #scheduler.step() 
            #scheduler을 설정했다면
            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
        #scheduler는 Learning rate를 설정을 도와줌
        #처음부터 끝가지 동일한 스텝을 밟는것보다 손실률이 클때는 크게, 손실률이 감소할때는 작게 찾아가게 설정
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')
        
        # Model Saved
        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), './saved/best_model.pth')
            print('Model Saved.')
            #기존의 mae보다 더 나은 모델이 있다면 해당 모델을 저장을 해서 모델을 완성!
            #그리고 이때 손실함수의 기준은 train 데이터가 아닌 validation 데이터 기준

In [13]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss
# 마찬가지로 만든 모델을 가지고 검증 데이터를 통해 과적합이 이루어지지 않았는가 검증!

## Run!!

* 옵티마이저  
데이터와 손실함수를 바탕으로 모델의 업데이트 방향을 결정
optima,Adadelta, Adagrad, Adam, SparseAdam, Adamax, ASGD, LBFGS, RMSProp, Rprop, SGD 등이 있다!
* 학습률 스케쥴러  
미리 지정한 횟수의 에포크를 지날때마다 학습률을 감소시켜 줍니다. 학습률 스케쥴러를 이용하면 학습 초기에는 빠른 학습을 진행하다가 전역 최소점 근처에 다다르면 학습률을 줄여서 최적점을 찾아갈 수 있도록 해줍니다. 학습률 스캐쥴러의 종류는 다음과 같음  
optim.li_scheduler.LambdaLR: 람다함수를 아용하여 함수의 결과를 학습률로 설정  
StepLR: 특정 단계마다 학습률을 감마 비율 만큼 감소 시킵니다.  
MultiStepLR: StepLR과 비슷하지만 특정 단계가 아닌 지정된 에포크에만 감마 비율로 감소  
ExponentialLR: 에포크마다 이전 학습률에 감마만큼 곱  

In [14]:
model = CNNRegressor().to(device)

optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#optimizer = torch.optim.RMSprop(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#optimizer = torch.optim.Adamax(model.parameters(), lr=CFG["LEARNING_RATE"])
#optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["LEARNING_RATE"], weight_decay=0.0001)

#scheduler = None
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,lr_lambda=lambda epoch:0.95**epoch)
train(model, optimizer, train_loader, vali_loader, scheduler, device)

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1] Train MAE : [59.46612] Validation MAE : [38.22414]

Model Saved.


  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [109] Train MAE : [10.48818] Validation MAE : [9.61717]



  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [110] Train MAE : [9.77894] Validation MAE : [17.66390]



## Inference 추론 모드  
추론(Inference)은 학습을 통해 만들어진 모델을 실제로 새로운 입력 데이터에 적용하여 결과를 내놓는 단계

In [15]:
def predict(model, test_loader, device):
    model.eval()
    # 학습할때 필요한 드롭아웃, 배치정규화등을 비활성화해서 추론만함
    model_pred = []
    with torch.no_grad():
    # 검증데이터에서 모델을 적용할때는 기울기를 추적하지 않음
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [16]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Score가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('./saved/best_model.pth')
model = CNNRegressor().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)

  0%|          | 0/58 [00:00<?, ?it/s]

카운트가 58개인 이유는 테스트데이터의 개수가 460개이고 이를 배치단위인 8로 나누면 57.5가 되는데 나머지 0.5를 안버리고 사용해서 58개  
마찬가지로 훈련데이터/검증데이터도 160개 40개인 이유

## Submission

In [17]:
submission = pd.read_csv('./sample_submission.csv')
submission['leaf_weight'] = preds
submission.to_csv('./submit.csv', index=False)